<div dir="rtl">

# ۳ – ارزیابی ساده‌ی RAG و جمع‌بندی نتایج

در این نوت‌بوک، مجموعه‌ای از سؤال‌های ارزیابی را آماده می‌کنید، سیستم RAG خود را روی آن‌ها اجرا می‌کنید، کیفیت پاسخ‌ها را بررسی می‌کنید و در پایان نتایج را به‌صورت کوتاه جمع‌بندی می‌کنید.

</div>


In [13]:
import os
import re
import json
import pickle
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize


In [ ]:
#DONE

In [ ]:
def load_questions_jsonl(path):
    items = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            items.append({
                "id": obj.get("id", ""),
                "question": obj.get("question", ""),
                "reference_answer": obj.get("reference_answer", ""),
                "doc_ids": obj.get("doc_ids", []),
            })
    return items

def tokenize_fa(s: str):
    s = "" if s is None else str(s)
    s = s.lower()
    s = re.sub(r"[^\w\u0600-\u06FF]+", " ", s)
    return [t for t in s.split() if t]

ART = "artifacts"
meta_loaded = pd.read_csv(os.path.join(ART, "chunks_meta.csv"))
index_sem_loaded = faiss.read_index(os.path.join(ART, "faiss_sem.index"))

with open(os.path.join(ART, "bm25.pkl"), "rb") as f:
    bm25_loaded = pickle.load(f)

with open(os.path.join(ART, "config.pkl"), "rb") as f:
    cfg = pickle.load(f)

semantic_model_loaded = SentenceTransformer(cfg["semantic_model"])

def _semantic_search(question, top_k):
    q = semantic_model_loaded.encode([question], convert_to_numpy=True).astype(np.float32)
    q = normalize(q, norm="l2").astype(np.float32)
    scores, idxs = index_sem_loaded.search(q, top_k)
    return idxs[0].tolist(), scores[0].tolist()

def _bm25_search(question, top_k):
    q_tokens = tokenize_fa(question)
    scores = np.asarray(bm25_loaded.get_scores(q_tokens), dtype=np.float32)
    if top_k >= len(scores):
        top_idx = np.argsort(-scores)
    else:
        top_idx = np.argpartition(scores, -top_k)[-top_k:]
        top_idx = top_idx[np.argsort(-scores[top_idx])]
    return top_idx.tolist(), scores[top_idx].tolist()

def retrieve(question, top_k=5, embedding_model="semantic", fusion_k=60, pool_mult=20):
    if embedding_model == "semantic":
        idxs, _ = _semantic_search(question, top_k)
        return meta_loaded.iloc[idxs]["text"].tolist()

    if embedding_model == "bm25":
        idxs, _ = _bm25_search(question, top_k)
        return meta_loaded.iloc[idxs]["text"].tolist()

    if embedding_model == "combined":
        pool_k = min(len(meta_loaded), top_k * pool_mult)
        sem_idxs, _ = _semantic_search(question, pool_k)
        bm_idxs, _ = _bm25_search(question, pool_k)

        fused = {}
        for r, i in enumerate(sem_idxs, start=1):
            fused[i] = fused.get(i, 0.0) + 1.0 / (fusion_k + r)
        for r, i in enumerate(bm_idxs, start=1):
            fused[i] = fused.get(i, 0.0) + 1.0 / (fusion_k + r)

        best = sorted(fused.items(), key=lambda x: x[1], reverse=True)[:top_k]
        idxs = [i for i, _ in best]
        return meta_loaded.iloc[idxs]["text"].tolist()

    raise ValueError("embedding_model باید 'semantic' یا 'bm25' یا 'combined' باشد")

def answer(question, top_k=5, embedding_model="semantic"):
    chunks = retrieve(question, top_k=top_k, embedding_model=embedding_model)
    return "\n\n".join(chunks)


In [ ]:
os.makedirs("evaluation", exist_ok=True)

questions = load_questions_jsonl("../evaluation/questions_template.jsonl")

modes = ["semantic", "bm25", "combined"]

results = []
for q in questions:
    qid = q["id"]
    question_text = q["question"]
    ref = q.get("reference_answer", "")
    doc_ids = q.get("doc_ids", [])

    for mode in modes:
        chunks = retrieve(question_text, top_k=3, embedding_model=mode)
        model_answer = answer(question_text, top_k=3, embedding_model=mode)

        results.append({
            "id": qid,
            "question": question_text,
            "embedding_model": mode,
            "model_answer": model_answer,
            "retrieved_chunks": chunks,
            "reference_answer": ref,
            "doc_ids": doc_ids,
        })

out_path = "evaluation/results.jsonl"
with open(out_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("saved:", out_path, "| rows:", len(results))


saved: evaluation/results.jsonl | rows: 45


In [ ]:
manual_accept = [
    False, True, True, True, True, True, False, False, False, True, True, True, True, True, True,
    False, True, True, True, True, True, True, True, True, True, True, True, True, True, True,
    True, True, True, True, True, True, True, True, True, False, False, False, False, True, True,
]

print("len(results):", len(results), "| len(manual_accept):", len(manual_accept))

models = ["semantic", "bm25", "combined"]
for m in models:
    idxs = [i for i, r in enumerate(results) if r["embedding_model"] == m]
    rate = np.mean([manual_accept[i] for i in idxs]) * 100
    print(f"{m}: {rate:.2f}%")


len(results): 45 | len(manual_accept): 45
semantic: 66.67%
bm25: 86.67%
combined: 86.67%


In [ ]:
good_idxs = [i for i, v in enumerate(manual_accept) if v][:3]
bad_idxs  = [i for i, v in enumerate(manual_accept) if not v][:3]

print("=== GOOD ANSWERS ===\n")
for i in good_idxs:
    r = results[i]
    print("Question:", r["question"])
    print("Model:", r["embedding_model"])
    print("Answer:")
    print(r["model_answer"][:800])
    print("\n" + "-"*60 + "\n")

print("\n=== BAD ANSWERS ===\n")
for i in bad_idxs:
    r = results[i]
    print("Question:", r["question"])
    print("Model:", r["embedding_model"])
    print("Answer:")
    print(r["model_answer"][:800])
    print("\n" + "-"*60 + "\n")


=== GOOD ANSWERS ===

Question: آخرین وضعیت قیمت دلار و ارز در بازار چیست؟
Model: bm25
Answer:
های تلخ بود. قیمت ها در بازار روز به روز رو به فزونی گذاشت و در حالی که دلارهای کمیاب برای واردات اقلام اساسی اختصاص می یافت، کالاهایی که وارد می شد در پاره از مواقع هیچ تناسبی با نیازهای بازار نداشت و از سوی دیگر کالاهای وارد شده در بازار با نرخ برابر ارز در بازار آزاد، به دست مصرف کننده نهایی می رسید. این کژکارکردی اقتصادی در نظام توزیع در کشور، باعث شد که هم ارز از ایران خارج شود و هم مردم ناچار باشند کالاهای مورد نیازشان را با قیمت های نجومی و برابر با نرخ دلار در بازار آزاد تهیه کنند. در چنین شرایطی در سال ۹۸ هم باردیگر دولت در لایحه بودجه از ارز ترجیحی دفاع و این سیاست رانت ساز را بار دیگر اجرایی کرد. یکی از کالاهایی که مشمول ارز ترجیحی ۴۲۰۰ تومانی است، نهاده ها و خوراک دام و طیور است که تاثیر مستقیم روی قیمت مرغ و اقلام دیگر مربوطه به این حوزه در بازار دارد. در این مدت بارها مرغدارا

------------------------------------------------------------

Question: آخرین وضعیت قیمت دلار و ارز در 

<div dir="rtl">

## خلاصه برای گزارش

در این بخش چند پاراگراف بنویسید و خلاصه کنید:

- از نظر شما، سیستم تقریباً به چند درصد سؤال‌ها پاسخ «قابل قبول» می‌دهد؟
- بزرگ‌ترین محدودیت‌ها و خطاهای تکرارشونده کدامند؟
- اگر زمان و منابع بیشتری در اختیار داشتید، چه بهبودهایی برای نسخه‌ی بعدی سیستم پیشنهاد می‌کنید؟  
  (برای مثال استفاده از مدل بهتر، بهبود روش چانک‌کردن متن‌ها، افزودن مراحل پیش‌پردازش یا تغییر شیوه‌ی تولید پاسخ.)

این متن می‌تواند مستقیماً در گزارش نهایی پروژه (به صورت PDF یا Markdown) استفاده شود.

</div>
